# Experiments with kernel machines

In this notebook we will use simple two-dimensional data sets to illustrate the behavior of the support vector machine and the Perceptron, when used with quadratic and RBF kernels.

## 1. Basic training procedure

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.svm import SVC
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14)

The directory containing this notebook should also contain two-dimensional data files, `data1.txt` through `data5.txt`. These files contain one data point per line, along with a label (either -1 or 1), like:
* `3 8 -1` (meaning that point `x=(3,8)` has label `y=-1`)

The next procedure, **learn_and_display_SVM**, loads one of these data sets, invokes `sklearn.SVC` to learn a classifier, and then displays the data as well as the boundary. It is invoked as follows:
* `learn_and_display_SVM(datafile, kernel_type, C_value, s_value)`

where
* `datafile` is one of `'data1.txt'` through `'data5.txt'` (or another file in the same format)
* `kernel_type` is either `'quadratic'` or `'rbf'`
* `C_value` is the setting of the soft-margin parameter `C` (default: 1.0)
* `s_value` (for the RBF kernel) is the scaling parameter `s` (default: 1.0)

In [ ]:
def learn_and_display_SVM(datafile, kernel_type='rbf', C_value=1.0, s_value=1.0):
    data = np.loadtxt(datafile)
    n,d = data.shape
    # Create training set x and labels y
    x = data[:,0:2]
    y = data[:,2]
    # Now train a support vector machine and identify the support vectors
    if kernel_type == 'rbf':
        clf = SVC(kernel='rbf', C=C_value, gamma=1.0/(s_value*s_value))
    if kernel_type == 'quadratic':
        clf = SVC(kernel='poly', degree=2, C=C_value, coef0=1.0)
    clf.fit(x,y)
    sv = np.zeros(n,dtype=bool)
    sv[clf.support_] = True
    notsv = np.logical_not(sv)
    # Determine the x1- and x2- limits of the plot
    x1min = min(x[:,0]) - 1
    x1max = max(x[:,0]) + 1
    x2min = min(x[:,1]) - 1
    x2max = max(x[:,1]) + 1
    plt.xlim(x1min,x1max)
    plt.ylim(x2min,x2max)
    # Plot the data points, enlarging those that are support vectors
    plt.plot(x[(y==1)*notsv,0], x[(y==1)*notsv,1], 'ro')
    plt.plot(x[(y==1)*sv,0], x[(y==1)*sv,1], 'ro', markersize=10)
    plt.plot(x[(y==-1)*notsv,0], x[(y==-1)*notsv,1], 'k^')
    plt.plot(x[(y==-1)*sv,0], x[(y==-1)*sv,1], 'k^', markersize=10)
    # Construct a grid of points and evaluate classifier at each grid points
    grid_spacing = 0.025
    xx1, xx2 = np.meshgrid(np.arange(x1min, x1max, grid_spacing), np.arange(x2min, x2max, grid_spacing))
    grid = np.c_[xx1.ravel(), xx2.ravel()]
    Z = clf.decision_function(grid)
    # Quantize the values to -1, -0.5, 0, 0.5, 1 for display purposes
    for i in range(len(Z)):
        Z[i] = min(Z[i],1.0)
        Z[i] = max(Z[i],-1.0)
        if (Z[i] > 0.0) and (Z[i] < 1.0):
            Z[i] = 0.5
        if (Z[i] < 0.0) and (Z[i] > -1.0):
            Z[i] = -0.5
    # Show boundary and margin using a color plot
    Z = Z.reshape(xx1.shape)
    plt.pcolormesh(xx1, xx2, Z, cmap=plt.cm.PRGn, vmin=-2, vmax=2)
    plt.show()

## 2. Experiments with the quadratic kernel

Let's try out SVM on some examples, starting with the quadratic kernel.

In [ ]:
C = 1
kernel_type = 'quadratic'

for file in ['data1.txt', 'data2.txt', 'data3.txt', 'data4.txt', 'data5.txt']:
    learn_and_display_SVM(file, kernel_type, C)

Also try `data2.txt` through `data5.txt`. Also try changing the value of `C` (the third parameter) to see how that affects the boundary and margin.

## 3. Experiments with the RBF kernel

Now experiment with the RBF kernel, on the same five data sets. This time there are two parameters to play with: `C` and `sigma`.

In [ ]:
C = 10.0
s_value = 10
kernel_type = 'rbf'

for file in ['data1.txt', 'data2.txt', 'data3.txt', 'data4.txt', 'data5.txt']:
    learn_and_display_SVM(file, kernel_type, C, s_value)


## 4. The kernel Perceptron

<font color="magenta">**For you to do:**</font> Implement the kernel Perceptron algorithm as specified in lecture. Your algorithm should allow both the quadratic and RBF kernel, and should follow roughly the same signature as the SVM routine above:
* `learn_and_display_Perceptron(datafile, kernel_type, s_value)`

Recall that the Perceptron algorithm does not always converge; you will need to explicitly check for this.

In [ ]:
### 
### Any auxiliary functions that you need
###

def quadratic_kernel(x1, x2):
    '''Returns quadratic kernel value for 2 vectors x1 and x2
    Inputs:
        x1, x2 = 2 vecors dimension d
    Outpus:
        k = kernel value'''
    k = (1 + x1 @ x2)**2
    return k


def rbf_kernel(x1, x2, s_value):
    '''Returns Radial Basis Function kernel value for 2 vectors x1 and x2
    Inputs:
        x1, x2 = 2 vecors dimension d
    Outpus:
        k = kernel value'''
    k = np.exp(-np.linalg.norm(x1 - x2) / s_value**2)
    return k


def quadratic_kernel_matrix(x):
    '''Returns quadratic kernel matrix K of a dataset x.
    Inputs:
        x = dataset matrix of size (n x d). n datapoints, d features.
    Outputs:
        K = kernel matrix. K[i,j] = (1 + x[i] @ x[j])**2'''
    n, d = x.shape
    K = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            K[i,j] = quadratic_kernel(x[i], x[j])
    return K


def rbf_kernel_matrix(x, s_value):
    '''Returns Radial Basis Function (RBF) kernel matrix K of a dataset x.
    Inputs:
        x = dataset matrix of size (n x d). n datapoints, d features.
    Outputs:
        K = kernel matrix. K[i,j] = np.exp(-np.linalg.norm(x[i] - x[j]) / s_value**2)'''
    n, d = x.shape
    K = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            K[i,j] = rbf_kernel(x[i], x[j], s_value)
    return K


def learn_and_display_Perceptron(datafile, kernel_type='rbf', s_value=1.0):
    ###
    ### Your code
    ###
    data = np.loadtxt(datafile)
    n,d = data.shape
    
    # Create training set x and labels y
    x = data[:,0:2]
    y = data[:,2]
    
    # Choose kernel
    if kernel_type == 'quadratic':
        K = quadratic_kernel_matrix(x)
    if kernel_type == 'rbf':
        K = rbf_kernel_matrix(x,s_value)
    
    # Perceptron
    N_iter = 1000 # number of passes through dataset
    alpha = np.zeros(n)
    b = 0
    alphas = np.zeros((n, N_iter))
    bs = []
    
    for t in range(N_iter):
        for i in range(n):
            if y[i] * (np.sum(alpha * y * K[i,:] + b)) <= 0:
                alpha[i] = alpha[i] + 1
                b = b + y[i]
        alphas[:, t] = alpha
        bs.append(b)
    
    # Identify points used by classifier
    sv = np.zeros(n,dtype=bool)
    sv[alpha > 0] = True
    notsv = np.logical_not(sv)
    
    # Determine the x1- and x2- limits of the plot
    x1min = min(x[:,0]) - 1
    x1max = max(x[:,0]) + 1
    x2min = min(x[:,1]) - 1
    x2max = max(x[:,1]) + 1
    plt.xlim(x1min,x1max)
    plt.ylim(x2min,x2max)
    
    # Plot the data points, enlarging those that are support vectors
    plt.plot(x[(y==1)*notsv,0], x[(y==1)*notsv,1], 'ro')
    plt.plot(x[(y==1)*sv,0], x[(y==1)*sv,1], 'ro', markersize=10)
    plt.plot(x[(y==-1)*notsv,0], x[(y==-1)*notsv,1], 'k^')
    plt.plot(x[(y==-1)*sv,0], x[(y==-1)*sv,1], 'k^', markersize=10)
    
    # Construct a grid of points and evaluate classifier at each grid points
    grid_spacing = 0.05
    xx1, xx2 = np.meshgrid(np.arange(x1min, x1max, grid_spacing), np.arange(x2min, x2max, grid_spacing))
    grid = np.c_[xx1.ravel(), xx2.ravel()]
    
    # Predict label of each point in grid
    Z = np.zeros(grid.shape[0])
    
    if kernel_type == 'quadratic':
        K_ = np.zeros(n)
        for j in range(grid.shape[0]):
            for i in range(n):
                K_[i] = quadratic_kernel(grid[j], x[i])
            Z[j] = np.sign(np.sum(alpha * y * K_ + b))
            
    if kernel_type == 'rbf':
        K_ = np.zeros(n)
        for j in range(grid.shape[0]):
            for i in range(n):
                K_[i] = rbf_kernel(grid[j], x[i], s_value)
            Z[j] = np.sign(np.sum(alpha * y * K_ + b))
    
    # Show boundary and margin using a color plot
    Z = Z.reshape(xx1.shape)
    plt.pcolormesh(xx1, xx2, Z, cmap=plt.cm.PRGn, vmin=-2, vmax=2)
    plt.show()
    
    
    return alpha, b, np.asmatrix(alphas), bs, Z

# Quadratic Kernel Perceptron

In [ ]:
kernel_type = 'quadratic'

for file in ['data1.txt', 'data2.txt', 'data3.txt', 'data4.txt', 'data5.txt']:
    alpha, b, alphas, bs, Z = learn_and_display_Perceptron(file, 
                                                       kernel_type=kernel_type, 
                                                       s_value=1.0)

# RBF Kernel Perceptron

In [ ]:
kernel_type = 'rbf'

for file in ['data1.txt', 'data2.txt', 'data3.txt', 'data4.txt', 'data5.txt']:
    alpha, b, alphas, bs, Z = learn_and_display_Perceptron(file, 
                                                       kernel_type=kernel_type, 
                                                       s_value=1)

<font color="magenta">Experiment with your routine, on the same five data sets.</font>